In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from datetime import datetime, date, timezone

# Short Example About Transformed Data

To extract data direct from github:

In [2]:
customers = pd.read_csv("https://raw.githubusercontent.com/dpalacioj/mlops-essentials/udemy-notes/section-02-feature-store/raw/customers.csv")

In [3]:
customers.head(2)

,customer_id,name,sex,state,age,is_married,active_since,event_time
0,C1,brooke williams,F,alabama,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z
1,C2,jim reese,M,oregon,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z


In [4]:
label_encoder = LabelEncoder()
min_max_scaler = MinMaxScaler()

In [5]:
customers.drop(columns = ['name', 'state'], inplace=True)

In [6]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   customer_id   10000 non-null  object
 1   sex           10000 non-null  object
 2   age           10000 non-null  int64 
 3   is_married    10000 non-null  bool  
 4   active_since  10000 non-null  object
 5   event_time    10000 non-null  object
dtypes: bool(1), int64(1), object(4)
memory usage: 400.5+ KB


In [7]:
customers['age'].value_counts(normalize=True, sort=True, ascending =False)

age
24    0.0168
42    0.0162
54    0.0161
81    0.0158
31    0.0158
       ...  
80    0.0118
91    0.0116
18    0.0112
65    0.0111
84    0.0105
Name: proportion, Length: 74, dtype: float64

In [8]:
bins = [18, 30, 40, 50, 60, 70, 90]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']
customers['age_range'] = pd.cut(customers['age'], bins, labels=labels, include_lowest=True)
customers.head(3)

,customer_id,sex,age,is_married,active_since,event_time,age_range
0,C1,F,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,70-plus
1,C2,M,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,70-plus
2,C3,M,40,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,30-39


The `cut` function is useful for going from a continuous variable to a categorical variable.

In [9]:
df_age_group = pd.get_dummies(customers['age_range'], prefix="age", dtype='int')
df_age_group.head(3)

,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,1,0,0,0,0


Once the categorical age features are obtained, they can be `concat` to the original dataframe

In [10]:
customers = pd.concat([customers, df_age_group], axis=1)
customers.head(3)

,customer_id,sex,age,is_married,active_since,event_time,age_range,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,C1,F,76,True,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,70-plus,0,0,0,0,0,1
1,C2,M,76,True,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,70-plus,0,0,0,0,0,1
2,C3,M,40,True,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,30-39,0,1,0,0,0,0


In [11]:
customers.drop(columns = ['age', 'age_range'], inplace=True) # Useless columns

Now, it is necesary to convert the `sex` feature from str to int. THe same applies to `is_married`

In [12]:
customers['sex'] = customers['sex'].map({"F":0, "M":1})
customers['is_married'] = customers['is_married'].astype('int') # Remember that 1=True and 0=False
customers.head()

,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,C1,0,1,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,0,0,0,0,0,1
1,C2,1,1,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,0,0,0,0,0,1
2,C3,1,1,2017-11-21 01:14:51,2024-05-02T05:39:10.967Z,0,1,0,0,0,0
3,C4,1,0,2016-11-16 04:44:55,2024-05-02T05:39:10.967Z,0,0,0,0,1,0
4,C5,1,0,2017-03-27 21:09:31,2024-05-02T05:39:10.968Z,0,0,0,0,1,0


In [13]:
customers['active_since'] = pd.to_datetime(customers['active_since'], format ="%Y-%m-%d %H:%M:%S")

In [14]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   customer_id   10000 non-null  object        
 1   sex           10000 non-null  int64         
 2   is_married    10000 non-null  int64         
 3   active_since  10000 non-null  datetime64[ns]
 4   event_time    10000 non-null  object        
 5   age_18-29     10000 non-null  int64         
 6   age_30-39     10000 non-null  int64         
 7   age_40-49     10000 non-null  int64         
 8   age_50-59     10000 non-null  int64         
 9   age_60-69     10000 non-null  int64         
 10  age_70-plus   10000 non-null  int64         
dtypes: datetime64[ns](1), int64(8), object(2)
memory usage: 859.5+ KB


In [18]:
customers['active_since'].dt.date

0       2019-03-09
1       2016-07-24
2       2017-11-21
3       2016-11-16
4       2017-03-27
           ...    
9995    2017-10-23
9996    2017-02-22
9997    2017-11-23
9998    2016-09-14
9999    2016-01-10
Name: active_since, Length: 10000, dtype: object

In [19]:
# To calculate the active time per each customer_id
def get_delta_days(datetime) -> int :
    today = date.today()
    delta = today - datetime.date()
    return delta.days

In [20]:
customers['n_days_active'] = customers['active_since'].apply(lambda x: get_delta_days(x))

In [21]:
customers.head(2)

,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2019-03-09 14:43:26,2024-05-02T05:39:10.965Z,0,0,0,0,0,1,2012
1,C2,1,1,2016-07-24 05:21:59,2024-05-02T05:39:10.966Z,0,0,0,0,0,1,2970


In [22]:
# To scale the data

customers['n_days_active'] = min_max_scaler.fit_transform(customers[['n_days_active']])
customers.drop("active_since", axis=1, inplace=True)

In [25]:
customers.to_csv("transformed/customers_transformed.csv")

# Transform Orders Data

In [27]:
orders = pd.read_csv("https://raw.githubusercontent.com/dpalacioj/mlops-essentials/udemy-notes/section-02-feature-store/raw/orders.csv")

In [28]:
orders.head(2)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time
0,O1,C9765,P11660,58.84,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z
1,O2,C3674,P6868,71.08,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z


In [30]:
orders['purchased_on'] = pd.to_datetime(orders['purchased_on'], format = '%Y-%m-%d %H:%M:%S')

In [31]:
# Apply again `get_delta_days`

orders['n_days_since_last_purchase'] = orders['purchased_on'].apply(lambda x: get_delta_days(x))

In [32]:
orders.head(2)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time,n_days_since_last_purchase
0,O1,C9765,P11660,58.84,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z,1339
1,O2,C3674,P6868,71.08,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z,1635


In [33]:
orders['n_days_since_last_purchase'] = min_max_scaler.fit_transform(orders[['n_days_since_last_purchase']])
orders.head(2)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time,n_days_since_last_purchase
0,O1,C9765,P11660,58.84,0,2021-01-10 10:31:27,2024-05-02T05:39:17.172Z,0.273256
1,O2,C3674,P6868,71.08,0,2020-03-20 16:47:36,2024-05-02T05:39:17.172Z,0.846899


In [34]:
orders['purchase_amount'] = min_max_scaler.fit_transform(orders[['purchase_amount']])

In [35]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   order_id                    100000 non-null  object        
 1   customer_id                 100000 non-null  object        
 2   product_id                  100000 non-null  object        
 3   purchase_amount             100000 non-null  float64       
 4   is_reordered                100000 non-null  int64         
 5   purchased_on                100000 non-null  datetime64[ns]
 6   event_time                  100000 non-null  object        
 7   n_days_since_last_purchase  100000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 6.1+ MB


In [36]:
orders['is_reordered'].value_counts()

is_reordered
1    66509
0    33491
Name: count, dtype: int64

In [37]:
orders.drop(columns="purchased_on", inplace=True)

In [38]:
orders.to_csv("transformed/orders_transformed.csv", index=False)